# Pulling metadata about MPs from WikiData 

In this file, I will run through how to obtain data from WikiData's API. I am using a Jypyter notebook because I think I will need to use python, or a mix of python and R. There used to be an SPARQL complier package in R which we could have used to write the request (then possibly combined with `httr` to dynamically pull the WikiData) but I could not get it to work with the latest version of R. 

It is also possible to manually pull MPs' tweets, however, it would be better - and more reproducible - to have code which can do it programmatically. 

The first code I will try is from [this blog post](https://sparqlwrapper.readthedocs.io/en/latest/main.html). I am having problems installing python packages so need to fix that first.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON 

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql" # I think this is the wrong place to put the http endpoint
)

sparql.setReturnFormat(JSON)

sparql.setQuery("""
    SELECT DISTINCT ?person ?personLabel ?seatLabel ?partyLabel ?countryLabel ?sexorgenderLabel ?dateofbirthLabel WHERE
  {
    ?person wdt:P31 wd:Q5 . ?person p:P39 ?ps .
    ?ps ps:P39 ?term . ?term wdt:P279 wd:Q16707842 .
    ?ps pq:P580 ?start . ?ps pq:P4100 ?party . ?ps pq:P768 ?seat .
    FILTER NOT EXISTS { ?ps pq:P582 ?end } .
    ?person wdt:P569 ?dateofbirth .
    ?person wdt:P21 ?sexorgender .
    ?seat wdt:P131+ ?country . ?country wdt:P31 wd:Q3336843 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language 'en'. }
  }
"""
)

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)
